# 03-2 트랜스포머를 사용한 텍스트 생성

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hm-dl/blob/main/03-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

코랩에서 이 노트북을 실행하려면 High-RAM CPU 런타임을 사용해야 합니다.

이 절의 코드를 실행하려면 `keras-nlp` 패키지와 허깅페이스 `transformers` 패키지를 위한 `tf-keras`를 설치해야 합니다.

In [ ]:
pip install -U tensorflow keras-nlp tf-keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py


In [ ]:
import keras
import keras_nlp

keras.__version__, keras_nlp.__version__

('3.3.3', '0.12.1')

In [ ]:
import keras
from keras import layers
import keras_nlp

In [ ]:
def make_causal_mask(seq_len):
    n_hori = keras.ops.arange(seq_len)
    n_vert = keras.ops.expand_dims(n_hori, axis=-1)
    mask = n_vert >= n_hori
    return mask

In [ ]:
def make_attention_mask(padding_mask):
    # padding_mask 크기가 (2, 5)라고 가정해 보죠.
    batch_size, seq_len = keras.ops.shape(padding_mask)
    # causal_mask 크기는 (5, 5)가 됩니다.
    causal_mask = make_causal_mask(seq_len)
    # 배치 차원을 추가해 (2, 5, 5)로 만듭니다.
    causal_mask = keras.ops.broadcast_to(causal_mask, (batch_size, seq_len, seq_len))
    # 브로드캐스팅을 위해 padding_mask 크기를 (2, 1, 5)로 만듭니다.
    padding_mask = keras.ops.expand_dims(padding_mask, axis=1)
    return keras.ops.minimum(causal_mask, padding_mask)

In [ ]:
class AttentionMask(keras.Layer):
    def call(self, padding_mask):
        return make_attention_mask(padding_mask)

## Gemma

### 젬마 구현하기

In [ ]:
from keras_nlp.src.models.gemma.gemma_attention import CachedGemmaAttention
from keras_nlp.src.models.gemma.rms_normalization import RMSNormalization

def gemma_decoder(x, padding_mask, num_query_heads, num_key_value_heads,
                  interm_dim, hidden_dim, head_dim):
    # 어텐션 마스크를 계산합니다.
    attention_mask = AttentionMask()(padding_mask)
    # 스킵 연결을 준비합니다.
    residual = x
    x = RMSNormalization()(x)
    # 멀티 헤드 어텐션을 통과합니다.
    gemma_attention = CachedGemmaAttention(head_dim=head_dim,
                                           num_query_heads=num_query_heads,
                                           num_key_value_heads=num_key_value_heads,
                                           dropout=0.0)
    x = gemma_attention(x, attention_mask)
    # 스킵 연결
    x = x + residual
    # 스킵 연결을 준비합니다.
    residual = x
    # 위치별 피드 포워드 네트워크
    x = RMSNormalization()(x)
    x1 = layers.Dense(interm_dim // 2, activation='gelu', use_bias=False)(x)
    x2 = layers.Dense(interm_dim // 2, use_bias=False)(x)
    x = x1 * x2
    x = layers.Dense(hidden_dim, use_bias=False)(x)
    # 스킵 연결
    x = x + residual
    return x

In [ ]:
# Gemma 2B
def make_gemma():
    vocab_size = 256000
    num_layers = 18
    num_query_heads = 8
    num_key_value_heads = 1
    interm_dim = 32768
    hidden_dim = 2048
    head_dim = 256

    token_ids = keras.Input(shape=(None,))
    padding_mask = keras.Input(shape=(None,))

    token_embedding_layer = keras_nlp.layers.ReversibleEmbedding(vocab_size, hidden_dim)
    x = token_embedding_layer(token_ids)
    x = layers.Lambda(lambda x: x * keras.ops.sqrt(hidden_dim))(x)

    for _ in range(num_layers):
        x = gemma_decoder(x, padding_mask, num_query_heads, num_key_value_heads,
                        interm_dim, hidden_dim, head_dim)

    x = RMSNormalization()(x)
    outputs = token_embedding_layer(x, reverse=True)
    return keras.Model(inputs=(token_ids, padding_mask),
                        outputs=(outputs))


model = make_gemma()
model.summary(line_length=100)

Model: "functional_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                ┃ Output Shape            ┃        Param # ┃ Connected to            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_64 (InputLayer) │ (None, None)            │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ reversible_embedding_32     │ (None, None, 256000)    │    524,288,000 │ input_layer_64[0][0],   │
│ (ReversibleEmbedding)       │                         │                │ rms_normalization_266[… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ lambda_5 (Lambda)           │ (None, None, 2048)      │              0 │ reversible_embedding_3… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ input_layer_65 (InputLayer) │ (None, None)            │              0 │ -                       │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ rms_normalization_230       │ (None, None, 2048)      │          2,048 │ lambda_5[0][0]          │
│ (RMSNormalization)          │                         │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ attention_mask_162          │ (None, None, None)      │              0 │ input_layer_65[0][0]    │
│ (AttentionMask)             │                         │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ cached_gemma_attention_108  │ (None, None, 2048)      │      9,437,184 │ rms_normalization_230[… │
│ (CachedGemmaAttention)      │                         │                │ attention_mask_162[0][… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ add_290 (Add)               │ (None, None, 2048)      │              0 │ cached_gemma_attention… │
│                             │                         │                │ lambda_5[0][0]          │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ rms_normalization_231       │ (None, None, 2048)      │          2,048 │ add_290[0][0]           │
│ (RMSNormalization)          │                         │                │                         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_446 (Dense)           │ (None, None, 16384)     │     33,554,432 │ rms_normalization_231[… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_447 (Dense)           │ (None, None, 16384)     │     33,554,432 │ rms_normalization_231[… │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ multiply_161 (Multiply)     │ (None, None, 16384)     │              0 │ dense_446[0][0],        │
│                             │                         │                │ dense_447[0][0]         │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ dense_448 (Dense)           │ (None, None, 2048)      │     33,554,432 │ multiply_161[0][0]      │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ add_291 (Add)               │ (None, None, 2048)      │              0 │ dense_448[0][0],        │
│                             │                         │                │ add_290[0][0]           │
├─────────────────────────────┼─────────────────────────┼────────────────┼─────────────────────────┤
│ rms_normalization_232       │ (None, None, 2048)      │    

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
del model

In [ ]:
import gc

gc.collect()

### 젬마 모델 사용하기

In [ ]:
gemma = keras_nlp.models.GemmaCausalLM.from_preset('gemma_2b_en')
gemma.summary()

Preprocessor: "gemma_causal_lm_preprocessor_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
sampler = keras_nlp.samplers.TopPSampler(p=0.8, seed=42)
gemma.compile(sampler=sampler)
gemma.generate('stay hungry, stay', max_length=20)

'stay hungry, stay foolish.  - steve jobs\nThomas Edison would have been proud of us'

In [ ]:
gemma.generate('봄이 오면', max_length=20)

'봄이 오면 전국에서 생강이 생산된다. 전국생강'

In [ ]:
gemma_pipe = pipeline("text-generation", model="beomi/gemma-ko-2b")
set_seed(42)
gemma_pipe('봄이 오면', max_length=20, truncation=True)

/anaconda/envs/py310_tf216_keras3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

[{'generated_text': '봄이 오면서 봄꽃이 피기 시작하면서 봄꽃축제가'}]

## 심화 예제

In [ ]:
# llama3
def make_llama3():
    vocab_size = 128256
    num_layers = 32
    num_query_heads = 32
    num_key_value_heads = 8
    interm_dim = 14336
    hidden_dim = 4096

    token_ids = keras.Input(shape=(None,))
    padding_mask = keras.Input(shape=(None,))

    token_embedding_layer = keras_nlp.layers.ReversibleEmbedding(vocab_size, hidden_dim,
                                                                tie_weights=False)
    x = token_embedding_layer(token_ids)

    for _ in range(num_layers):
        x = llama_decoder(x, padding_mask, num_query_heads, num_key_value_heads,
                        interm_dim, hidden_dim)

    x = LlamaLayerNorm()(x)
    outputs = token_embedding_layer(x, reverse=True)
    return keras.Model(inputs=(token_ids, padding_mask),
                       outputs=(outputs))


model = make_llama3()
model.summary(line_length=100)

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reversible_embeddi… │ (None, None,      │ 1,050,673… │ input_layer_20[0… │
│ (ReversibleEmbeddi… │ 128256)           │            │ llama_layer_norm… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_21      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ llama_layer_norm_3… │ (None, None,      │      4,096 │ reversible_embed… │
│ (LlamaLayerNorm)    │ 4096)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask_175  │ (None, None,      │          0 │ input_layer_21[0… │
│ (AttentionMask)     │ None)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ llama_attention_160 │ (None, None,      │ 41,943,040 │ llama_layer_norm… │
│ (LlamaAttention)    │ 4096)             │            │ attention_mask_1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_345 (Add)       │ (None, None,      │          0 │ llama_attention_… │
│                     │ 4096)             │            │ reversible_embed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ llama_layer_norm_3… │ (None, None,      │      4,096 │ add_345[0][0]     │
│ (LlamaLayerNorm)    │ 4096)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_506 (Dense)   │ (None, None,      │ 58,720,256 │ llama_layer_norm… │
│                     │ 14336)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_507 (Dense)   │ (None, None,      │ 58,720,256 │ llama_layer_norm… │
│                     │ 14336)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_160        │ (None, None,      │          0 │ dense_506[0][0],  │
│ (Multiply)          │ 14336)            │            │ dense_507[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_508 (Dense)   │ (None, None,      │ 58,720,256 │ multiply_160[0][… │
│                     │ 4096)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_346 (Add)       │ (None, None,      │          0 │ dense_508[0][0],  │
│                     │ 4096)             │            │ add_345[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ llama_layer_norm_3… │ (None, None,      │      4,096 │ add_346[0][0]     │
│ (LlamaLayerNorm)    │ 4096)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask_176  │ (None, None,      │          0 │ input_layer_21[0… │
│ (AttentionMask)     │ None)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ llama_attention_161 │ (None, None,      │ 41,943,040 │ llama_layer_norm… │
│ (LlamaAttention)    │ 4096)             │            │ attention_mask_1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_347 (Add)       │ (None, None,      │          0 │ llama_attention_

 Total params: 8,030,261,248 (29.92 GB)

 Trainable params: 8,030,261,248 (29.92 GB)

 Non-trainable params: 0 (0.00 B)